# Representing Data as Graph

In [ ]:
#Imports
import sys
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
"""
A function for reading sdf files
"""

from typing import Optional, Tuple, List


def parse_sdf(
    filename: str, include_hydrogen: Optional[bool] = False
) -> Tuple[List, List]:
    """
    Read an sdf file and return the atom and bond info

    Parameters
    ----------
    filename : str
        The name of the file to analyze
    include_hydrogen : Optional[bool]
        Controls whether information about hydrogens is returned.

    Returns
    -------
    elements : list
        A list of elements
    bonds: list
        A list of tuples where the first two numbers represent the atom indices
        and the third represents the bond order.
    """

    with open(filename) as f:
        data = [x.strip() for x in f.readlines()]

    num_atoms = int(data[3].split()[0])
    num_bonds = int(data[3].split()[1])

    atom_end = 4 + num_atoms
    bond_end = 4 + num_atoms + num_bonds

    atoms = data[4:atom_end]
    bonds = data[atom_end:bond_end]

    elements = [x.split()[3] for x in atoms]

    bond_pairs = [
        (int(x.split()[0]), int(x.split()[1]), int(x.split()[2])) for x in bonds
    ]

    if include_hydrogen is not True:
        bond_pairs = [
            x
            for x in bond_pairs
            if elements[x[0] - 1] != "H" and elements[x[1] - 1] != "H"
        ]
        elements = [x for x in elements if x != "H"]

    # Fix so that bond indices use python indexing
    bond_pairs = [(x[0] - 1, x[1] - 1, x[2]) for x in bond_pairs]

    return elements, bond_pairs

In [ ]:
"""
Functions to create a graph representation of our molecule and save it
"""

#Dictionary for CPK Coloring
COLOR = {
        "H" : "white",
        "O" : "red",
        "N" : "blue",
        "C": "gray"
    }

def create_graph(nodes: list[tuple], edges: list[tuple]) -> nx.Graph:
    """
    Creating a graph from our atoms and bond pairs extracted from our element file

    Parameters
    ----------
    nodes : list[tuple]
        Our atoms extracted from an element file
    edges : list[tuple]
        The bond pairs in our element
    Returns
        Our element represented as a graph of nodes and edges
    -------
    """
    #Create an instance of our networkx graph
    g = nx.Graph()
    
    #Extract our atom positions from our edges list
    positions = [e[0:2] for e in edges]
    g.add_edges_from(positions)

    #Create a dictionary of our atoms to label our graph with
    labels = {}
    for i in range(len(g.nodes)):
        labels[i] = nodes[i]

    #Set each atom to its corresponding color
    for i in range(len(g.nodes)):
        atom_color = COLOR.get(labels[i])
        g.add_node(i, color=atom_color)
    colors = nx.get_node_attributes(g, "color")

    #Create our graph
    pos = nx.spring_layout(g)
    nx.draw_networkx(g, pos, with_labels=False, node_color=[colors[n] for n in g.nodes()], edgecolors="black", node_size=500, edge_color="black")
    nx.draw_networkx_labels(g, pos, labels=labels)

    return g

def visualize_molecule(graph: nx.Graph):
    """
    Creating the image of our graph

    Parameters
    ----------
    graph: nx.Graph
        Our graph created from create_graph()
    Returns
        Our graph represented in a figure
    -------
    """
    #Save our figure based on the molecule file we are passing through
    plt.savefig(f"{sys.argv[1]}_graph.png")


nodes,edges = parse_sdf(f"{sys.argv[1]}.sdf")
graph = create_graph(nodes, edges)
visualize_molecule(graph)



In [ ]:
def count_rings(graph: nx.Graph) -> tuple[int, list[int]]:
    """
    Counting the number of rings

    Parameters
    ----------
    graph: nx.Graph
        Our graph created from create_graph()
    Returns
        The number of rings found in our molecule
    -------
    """

    #Rings are represented as cycles in our graph
    cycles = list(nx.cycle_basis(graph))
    atoms = graph.number_of_nodes()

    return len(cycles), atoms


nodes,edges = parse_sdf(f"{sys.argv[1]}.sdf")
graph = create_graph(nodes, edges)
rings, atoms = count_rings(graph)

print(f"{sys.argv[1]} has {rings} rings and {atoms} atoms")